In [1]:
from pyspark.sql import SparkSession
import os

# Point to Java 17
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home"

spark = (
    SparkSession.builder
    .appName("S3 Parquet Reader")
    .config("spark.jars", "/Users/billyheidel/spark-jars/hadoop-aws-3.4.1.jar,/Users/billyheidel/spark-jars/bundle-2.25.52.jar")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/01 21:56:28 WARN Utils: Your hostname, Billys-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 10.14.43.78 instead (on interface en0)
25/08/01 21:56:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/08/01 21:56:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# Configure AWS credentials
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.access.key", os.environ["AWS_ACCESS_KEY_ID"])
hadoop_conf.set("fs.s3a.secret.key", os.environ["AWS_SECRET_ACCESS_KEY"])
hadoop_conf.set("fs.s3a.endpoint", "s3.amazonaws.com")
hadoop_conf.set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")

# Load Parquet GZ file
df_parquet = spark.read.parquet(
    "s3a://billy-heidel-test-bucket/USDA_FoodCentral/Normalized/usda_data_cleaned/branded_foods_json/1751455506.84567.653734584a.parquet.gz"
)

# Show schema and some rows
df_parquet.printSchema()
df_parquet.show(5, truncate=False)

25/08/01 21:56:31 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


root
 |-- source_file: string (nullable = true)
 |-- _dlt_load_id: string (nullable = true)
 |-- _dlt_id: string (nullable = true)



+--------------------------------------------------+----------------+--------------+
|source_file                                       |_dlt_load_id    |_dlt_id       |
+--------------------------------------------------+----------------+--------------+
|FoodData_Central_branded_food_json_2025-04-24.json|1751455506.84567|PWkVSurSJrCBIg|
+--------------------------------------------------+----------------+--------------+



25/08/01 22:18:49 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 177991 ms exceeds timeout 120000 ms
25/08/01 22:18:49 WARN SparkContext: Killing executors is not supported by current scheduler.
25/08/01 22:18:56 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:132)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$